# (Ford Go Bike Exploration)
## by (Moamen Mohamed Mohasseb)

## Preliminary Wrangling

> Briefly introduce your dataset here.

# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
%matplotlib inline

### What is the structure of your dataset?

> Number of rows 183412 , Data columns  16 columns.

### What is/are the main feature(s) of interest in your dataset?

> what is the relation between gender and bike rental? what is the average duration? what is the average age that this bike? 

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Member_gender , user_type  trip duration ,duration_sec ,start_station_name and member_birth_year.

In [ ]:
#load data
bike_df=pd.read_csv("201902-fordgobike-tripdata.csv")
bike_df.head()

In [ ]:
bike_df.info()

In [ ]:
bike_df.describe()

In [ ]:
bike_df.isnull().sum()

In [ ]:
bike_df[bike_df.start_station_id.isna()]

In [ ]:
bike_df.query("start_station_latitude == 37.40 and start_station_id== 'NaN' ")

In [ ]:
bike_df['member_gender'].value_counts()

In [ ]:
(bike_df[bike_df['member_birth_year']<1940])

## Data wrangling

In [ ]:
bike_df_copy=bike_df.copy()

In [ ]:
#drop all null value

bike_df_copy.dropna(inplace=True)

In [ ]:
#change data type  of  start_time & end_time to date time
bike_df_copy['start_time']=pd.to_datetime(bike_df_copy['start_time'])
bike_df_copy['end_time']=pd.to_datetime(bike_df_copy['end_time'])

In [ ]:
#change data type  of end_station_id , start_station_id , member_birth_year
bike_df_copy['member_birth_year']=bike_df_copy['member_birth_year'].apply(int)
bike_df_copy['start_station_id']=bike_df_copy['start_station_id'].apply(int)
bike_df_copy['end_station_id']=bike_df_copy['end_station_id'].apply(int)

In [ ]:
bike_df_copy.info()

In [ ]:
#isolate day and month  of  start_time & end_time 
bike_df_copy['day']=bike_df_copy.start_time.dt.strftime('%a')
bike_df_copy['start_hour']  = bike_df_copy.start_time.dt.hour
bike_df_copy['age']  =2020-bike_df_copy.member_birth_year
bike_df_copy.sample(5)

In [ ]:
bike_df_copy.describe()

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

In [ ]:
# plot relation between member gender and number of rides
member_gender=bike_df_copy['member_gender'].value_counts()
plt.pie(member_gender.values,labels=member_gender.index,counterclock=False,autopct='%.0f%%',textprops={'size': 'larger'},
                                          shadow=True, radius=1.5);

> As we can see men is the majorty of users with 75% of rides numbers and womens only 23%

In [ ]:
# plot member ages and set limit between 20 and 80 years as below and above this value consider outliers or typo errors
bin_size=np.arange(0,bike_df_copy.age.max()+1,1)
plt.figure(figsize=[8,5])
plt.hist(data=bike_df_copy,x='age',bins=bin_size)
plt.xlabel("Member age")
plt.ylabel("Number of Rides")
plt.xlim([20,80])
plt.show()

> The plot of member ages is right-skewed its normal so we  set limit between 20 and 80 years old as below and above this value consider outliers or typo errors as   some members has 140,120,100 years old.

In [ ]:
#the relation between day and Number of rides , between month and Number of ride
day_order=['Sun','Mon','Tue','Wed','Thu','Fri','Sat']
#month_order=['',]
fig, ax=plt.subplots(nrows=2,figsize=[8,8])
default_color=sb.color_palette()[5]
sb.countplot(data=bike_df_copy,x='day',color=default_color,ax=ax[0],order=day_order)
sb.countplot(data=bike_df_copy,x='start_hour',color=default_color,ax=ax[1])

> * As we can see Number of rides in week ends is less than other work days
> * It seems that many users use bikes to go and return back from work as we notice increase in rides number from 7 to 9 Am
when users go to work usually and increase also from 4 to 6 PM when they return back to home.
> * also increase rides number in work days than week end support this observation.

In [ ]:
# plot member ages and set limit between 20 and 80 years as below and above this value consider outliers or typo errors
explode=(0,0.1)
plt.pie(bike_df_copy.user_type.value_counts().values,labels=bike_df_copy.user_type.value_counts().index,counterclock=False,autopct='%.0f%%',textprops={'size': 'larger'},
                                          shadow=True, radius=1,explode=explode);
plt.title("Relation between Number of rides and User Type",bbox={'facecolor':'0.8', 'pad':5})
plt.show()


> the majorty of user type is Subscriber as 91% of them and ordinary customer is just 9%

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> * The plot of member ages is right-skewed its normal so we set limit between 20 and 80 years old as below and above this value consider outliers or typo errors as some members has 140,120,100 years old.
> * As we can see Number of rides in week ends is less than other work days
It seems that many users use bikes to go and return back from work as we notice increase in rides number from 7 to 9 Am when users go to work usually and increase also from 4 to 6 PM when they return back to home.
also increase rides number in work days than week end support this observation.
> * the majorty of user type is Subscriber as they about 91% of them and ordinary customer is just 9% .

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> The plot of member ages is right-skewed , we use day and hour from start time column as majorty of trips happen in the same day (so i didnot use end time column).i create new column age as compare ages better than using birth date.

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

In [ ]:
type_order=['Male','Female','Other']
plt.figure(figsize=[8,12])
plt.subplot(2,1,1)
sb.countplot(data=bike_df_copy,x='day',hue='member_gender',palette='Blues',hue_order=type_order)
plt.legend(ncol=3)
ax=plt.subplot(2,1,2)
sb.countplot(data=bike_df_copy,x='start_hour',hue='member_gender',palette='Reds',hue_order=type_order)
ax.legend(ncol = 1)

In [ ]:
# plot comparison  between customer and subscriber  with user_type 
g = sb.FacetGrid(data=bike_df_copy, col='user_type', col_wrap=2, height =4 )
g.map(sb.countplot, 'member_gender', palette='Blues');
g.set_titles('{col_name}');

> * #### Males is the majority of subscriber.
> * #### Other is the majority of customer. 

In [ ]:
#relation between age and trip duration for large duration only 
plt.scatter(data=bike_df_copy,y='duration_sec',x='age')
plt.xlim([20,80])
plt.ylim([20000,80000])
plt.xlabel("Members Age")
plt.ylabel("Trip duration in second")
plt.title("relation between Age and Trip duration",bbox={'facecolor':'0.8', 'pad':4})

> #### Relation between age and trip duration for large duration only as we can see from 20 years old to 50 years old get long Trip duration than old people  which is normal , we cannot compare ages with short trip duration as all ages has no difference #### 

In [ ]:
#relation between age and trip duration and days of week
sb.barplot(data=bike_df_copy, x='day', y='duration_sec', color=sb.color_palette()[0],order=day_order);
plt.xlabel('Day of Week');
plt.ylabel('Trip Duration in second');

>  #### We observe that weekend has large trip duration than work day###

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

In [ ]:
bike_df_copy.info()